In [1]:
import warnings
warnings.filterwarnings('ignore')
import time
import pandas as pd
from tqdm import tqdm 
pd.options.display.max_columns = 100

__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE

In [2]:
train = pd.read_pickle('../../data/ridge/train_xgb_baseline.pkl')
test = pd.read_pickle('../../data/ridge/test_xgb_baseline.pkl')
selected_1 = pd.read_pickle('../../data/ridge/selected_agg1.pkl')
selected_2 = pd.read_pickle('../../data/ridge/selected_agg2.pkl')
train = train.merge(selected_1,on='instance_id',how='inner')
train = train.merge(selected_2,on='instance_id',how='inner')
print(train.shape)
test = test.merge(selected_1,on='instance_id',how='inner')
test = test.merge(selected_2,on='instance_id',how='inner')
print(test.shape)

(1001650, 381)
(40024, 381)


In [3]:
not_feats = ['instance_id', 'click', 'time_day', 'time']
# cat_cols = test.columns
# cat_cols = [col for col in cat_cols if 'count' not in col and col not in not_feats]

# num_cols = [col for col in test.columns if 'count' in col]
start_feats = list(set(test.columns) - set(not_feats))

In [4]:
# append_count = pd.read_pickle(FILE.X_fe_agg_count_formater.value.format(2))
# print(append_count.shape)
# append_timeNext = pd.read_pickle(FILE.X_fe_agg_time_next_formater.value.format(2))
# print(append_timeNext.shape)
# append_timeCount = pd.read_pickle(FILE.X_fe_agg_time_count_formater.value.format(2))
# print(append_timeCount.shape)

# X_fe = append_count.merge(append_timeNext,on='instance_id',how='inner')
# X_fe = X_fe.merge(append_timeCount,on='instance_id',how='inner')
X_fe = pd.read_pickle('agg2_temp.pkl')
print(X_fe.shape)

# fillna
cols = list(set(X_fe.columns)-set(['instance_id']))
for col in tqdm(cols):
    if 'to_n_previous' in col:
        X_fe[col] = X_fe[col].fillna(1)
    elif 'to_n_next' in col:
        X_fe[col] = X_fe[col].fillna(-1)
    else:
        X_fe[col] = X_fe[col].fillna(0)


  3%|▎         | 98/3826 [00:00<00:03, 975.37it/s]

(1041674, 3827)


100%|██████████| 3826/3826 [00:03<00:00, 996.33it/s] 


In [5]:
# train = train.merge(X_fe,how='inner',on='instance_id')
# print(train.shape)

# test = test.merge(X_fe,how='inner',on='instance_id')
# print(test.shape)


In [5]:
train_order = train[['instance_id']].copy()
train_order['ori_order'] = train_order.index
test_order = test[['instance_id']].copy()
test_order['ori_order'] = test_order.index

X_fe_order = X_fe[['instance_id']].copy()
X_fe_order['fe_order'] = X_fe_order.index
X_fe_train = X_fe_order.merge(train_order,on='instance_id',how='inner')
X_fe_train = X_fe_train.sort_values('ori_order',ascending=True)
X_fe_test = X_fe_order.merge(test_order,on='instance_id',how='inner')
X_fe_test = X_fe_test.sort_values('ori_order',ascending=True)
index_order_train = X_fe_train.fe_order.values
index_order_test = X_fe_test.fe_order.values

X_fe_train = X_fe.iloc[index_order_train].copy()
print(X_fe_train.shape)

X_fe_test = X_fe.iloc[index_order_test].copy()
print(X_fe_test.shape)

assert np.sum(X_fe_train.instance_id.values != train.instance_id.values) == 0
assert np.sum(X_fe_test.instance_id.values != test.instance_id.values) == 0

for col in tqdm(X_fe_train.columns):
    if col == 'instance_id':
        continue
    else:
        train[col] = X_fe_train[col].values
        test[col] = X_fe_test[col].values
print(train.shape)
print(test.shape)

(1001650, 3827)


  1%|          | 40/3827 [00:00<00:10, 373.25it/s]

(40024, 3827)


100%|██████████| 3827/3827 [01:55<00:00, 33.17it/s]

(1001650, 4207)
(40024, 4207)


In [6]:
all_feat = list(set(test.columns) - set(not_feats))

In [7]:
# import copy
# train_count = copy.copy(train)
# test_count = copy.copy(test)
# train_count = train_count.fillna(-99999)
# test_count = test_count.fillna(-99999)
# train_count["test"]=0
# test_count["test"]=1
# temp_df = pd.concat([train_count,test_count],axis=0)
# for col in tqdm(cat_cols):
#     temp_df.loc[:,col]=temp_df.loc[:,col].astype('category')
# train_count=temp_df[temp_df["test"]==0].iloc[:,:-1]
# test_count=temp_df[temp_df["test"]==1].iloc[:,:-1]

# for col in tqdm(cat_cols):
# #     counts = train_count.iloc[:,i].value_counts()
#     counts = train_count[col].value_counts()
#     counts = counts.sort_index()
#     counts = counts.fillna(0)
#     counts += np.random.rand(len(counts))/1000
#     train_count.loc[:,col].cat.categories = counts
#     test_count.loc[:,col].cat.categories = counts
    
# for col in cat_cols:
#     train_count[col] = train_count[col].astype(float)
#     test_count[col] = test_count[col].astype(float)
    
# train_count[num_cols] = train_count[num_cols].fillna(0)

In [7]:
train_p = train
X = train_p.drop(columns=not_feats)
y = train_p['click']

In [8]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score
clf = RidgeClassifier()

In [9]:

# start_feats = ['model', 'user_id', 'count_model', 'count_city', 'amt_usertags', 'osv', 'inner_slot_id']
feats_to_test = [feat for feat in all_feat if feat not in start_feats]

In [11]:
fe_path = '../../data/ridge/selected_agg2.pkl'

init_feats = start_feats
init_score = cross_val_score(clf, X[init_feats], y, cv=5, scoring='roc_auc')
init_score = init_score.mean()
print('init auc: ', init_score)

try:
    X_save = pd.read_pickle(fe_path)
except:
    X_save = pd.DataFrame()
for i, feat in enumerate(feats_to_test):
    scores = cross_val_score(clf, X[init_feats+[feat]], y, cv=5, scoring='roc_auc')
    auc = scores.mean()
    if auc > init_score:
        init_score = auc
        init_feats = init_feats+[feat]
        print('#{}/{} | YES! {} [{}] {:.6f}'.format(i, len(feats_to_test), feat, len(init_feats), init_score))
        X_save[feat] = X_fe[col].values
        if 'instance_id' not in X_save.columns:
            X_save['instance_id'] = X_fe['instance_id'].values
        X_save.to_pickle(fe_path)
        
    else:
        print('#{}/{} | NO!! {} [{}] {:.6f}'.format(i, len(feats_to_test), feat, len(init_feats), init_score))
    

init auc:  0.7387881839795222
#0/3826 | YES! orderid+carrier_count [378] 0.738794
#1/3826 | NO!! campaign_id+osv_count_in_previous_n_time_unit_1hour [378] 0.738794
#2/3826 | NO!! model+creative_has_deeplink_count_in_previous_n_time_unit_24hour [378] 0.738794
#3/3826 | NO!! time_hour+inner_slot_id_time_to_n_next [378] 0.738794
#4/3826 | NO!! creative_size+creative_id_time_to_n_previous [378] 0.738794
#5/3826 | NO!! model+creative_is_jump_count_in_next_n_time_unit_6hour [378] 0.738794
#6/3826 | NO!! app_id+os_name_cumulative_count [378] 0.738794
#7/3826 | YES! app_cate_id+city_time_to_n_next [379] 0.738814
#8/3826 | YES! app_cate_id+osv_time_to_n_previous [380] 0.738815
#9/3826 | NO!! advert_industry_inner+advert_id_count_in_previous_n_time_unit_6hour [380] 0.738815
#10/3826 | NO!! advert_industry_inner+app_id_count_in_next_n_time_unit_6hour [380] 0.738815
#11/3826 | NO!! adid+inner_slot_id_count_in_previous_n_time_unit_24hour [380] 0.738815
#12/3826 | NO!! creative_is_download+make_coun

KeyboardInterrupt: 

In [12]:
saving = X_fe[feats_to_test[18:]].copy()
saving['instance_id'] = X_fe['instance_id'].values
saving.to_pickle('agg2_temp.pkl')

KeyboardInterrupt: 